In [3]:
import requests
import pandas as pd

# Get Metadata

We can use the autocomplete API to get the list of available tables and fields to build our CSV exports.

In [17]:
autocomplete_url = "https://dragalialost.gamepedia.com/api.php?action=cargoqueryautocomplete&format=json&search="
result_key = "cargoqueryautocomplete"

## Tables

In [25]:
tables = requests.get(autocomplete_url).json()[result_key]
pd.DataFrame(tables)

,main_table
0,ABRCharacter
1,ABRDragon
2,Abilities
3,AbilityGroup
4,AbilityLimitedGroup
...,...
56,WyrmprintRarity
57,WyrmprintTrade
58,Wyrmprints
59,Wyrmprintsv1


## Fields

In [51]:
def get_fields(table):
    table_name = table["main_table"]
    fields = requests.get(autocomplete_url, params={"tables": table_name}).json()[result_key]
    return {**table, **{"fields": fields}}

In [60]:
import concurrent.futures
import time

start = time.time()

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(get_fields, table) for table in tables]
    tables_and_fields = [f.result() for f in futures]
    
print(f"Elapsed Time:{time.time() - start}")

Elapsed Time:0.8138711452484131


In [61]:
pd.DataFrame(tables_and_fields)

,main_table,fields
0,ABRCharacter,"[ABRCharacter.Id, ABRCharacter.Portrait, ABRCh..."
1,ABRDragon,"[ABRDragon.Id, ABRDragon.Portrait, ABRDragon.N..."
2,Abilities,"[Abilities.Id, Abilities.GenericName, Abilitie..."
3,AbilityGroup,"[AbilityGroup.Id, AbilityGroup.GroupName, Abil..."
4,AbilityLimitedGroup,"[AbilityLimitedGroup.Id, AbilityLimitedGroup.I..."
...,...,...
56,WyrmprintRarity,"[WyrmprintRarity.Id, WyrmprintRarity.MaxLimitL..."
57,WyrmprintTrade,"[WyrmprintTrade.Id, WyrmprintTrade.AbilityCres..."
58,Wyrmprints,"[Wyrmprints.Id, Wyrmprints.BaseId, Wyrmprints...."
59,Wyrmprintsv1,"[Wyrmprintsv1.Id, Wyrmprintsv1.BaseId, Wyrmpri..."


# Get CSVs

Now that we have the metadata, let's just grab all the data so we can mess around with it.

Here's an example query:

```
https://dragalialost.gamepedia.com/Special:CargoExport?tables=Abilities%2C&&fields=Abilities.Id%2C+Abilities.GenericName%2C&&order+by=%60cargo__Abilities%60.%60Id%60%2C%60cargo__Abilities%60.%60GenericName%60&limit=100&format=json
```

Transliterated into stuff you'd type:

```
https://dragalialost.gamepedia.com/Special:CargoExport?tables=Abilities, &fields=Abilities.Id, Abilities.GenericName,&&order by=`cargo__Abilities`.`Id`,`cargo__Abilities`.`GenericName`&limit=100&format=json
```

Finally, we'll separate the whole thing into the param dictionary:

```python
url="https://dragalialost.gamepedia.com/Special:CargoExport"
formatting_params = {
    "format": "json",
    "limit": "100" # We'll take this out later
}
data_params = {
    "tables": "Abilities" # our main_table column
    "fields": "Abilities.Id,Abilities.GenericName" # Comma separated list of our fields column. We can probably leave out the table names; we're not doing a join. btw spaces are optional
    "order by": "`cargo__Abilities`.`id`,`cargo__Abilities`.`GenericName`" # We can ask the server to order the fields for us though I'm unsure if the server can really handle that
}
params = {**formatting_params, **data_params}
```

Let's see about our order_by fields ... I'll be committing these files to github so we'll _really_ appreciate ordering when we do a diff. May as well ask the server to do it for us.

## Create OrderBy Fields

In [69]:
for table in tables_and_fields:
    default_ids = [i for i in table['fields'] if 'Id' in i]
    backup_ids = [i for i in table['fields'] if 'ID' in i] # I suspect these are psuedo ID fields for pagination, not sure we can trust them not to change
    order_by = default_ids if default_ids else backup_ids
    table["order_by"] = order_by

In [70]:
pd.DataFrame(tables_and_fields)

,main_table,fields,order_by
0,ABRCharacter,"[ABRCharacter.Id, ABRCharacter.Portrait, ABRCh...","[ABRCharacter.Id, ABRCharacter.WeaponTypeId]"
1,ABRDragon,"[ABRDragon.Id, ABRDragon.Portrait, ABRDragon.N...",[ABRDragon.Id]
2,Abilities,"[Abilities.Id, Abilities.GenericName, Abilitie...","[Abilities.Id, Abilities.AbilityLimitedGroupId..."
3,AbilityGroup,"[AbilityGroup.Id, AbilityGroup.GroupName, Abil...",[AbilityGroup.Id]
4,AbilityLimitedGroup,"[AbilityLimitedGroup.Id, AbilityLimitedGroup.I...",[AbilityLimitedGroup.Id]
...,...,...,...
56,WyrmprintRarity,"[WyrmprintRarity.Id, WyrmprintRarity.MaxLimitL...",[WyrmprintRarity.Id]
57,WyrmprintTrade,"[WyrmprintTrade.Id, WyrmprintTrade.AbilityCres...","[WyrmprintTrade.Id, WyrmprintTrade.AbilityCres..."
58,Wyrmprints,"[Wyrmprints.Id, Wyrmprints.BaseId, Wyrmprints....","[Wyrmprints.Id, Wyrmprints.BaseId, Wyrmprints...."
59,Wyrmprintsv1,"[Wyrmprintsv1.Id, Wyrmprintsv1.BaseId, Wyrmpri...","[Wyrmprintsv1.Id, Wyrmprintsv1.BaseId, Wyrmpri..."
